# Práctica 2: Algoritmos metaheurísticos

## Sistemas Inteligentes

### Curso académico 2022-2023

#### Profesorado:

* Juan Carlos Alfaro Jiménez (`JuanCarlos.Alfaro@uclm.es`)
* Guillermo Tomás Fernández Martín (`Guillermo.Fernandez@uclm.es`)
* María Julia Flores Gallego (`Julia.Flores@uclm.es`)
* Ismael García Varea (`Ismael.Garcia@uclm.es`)
* Luis González Naharro (`Luis.GNaharro@uclm.es`)
* Aurora Macías Ojeda (`Profesor.AMacias@uclm.es`)
* Marina Sokolova Sokolova (`Marina.Sokolova@uclm.es`)

## 0. Preliminares

Antes de comenzar con el desarrollo de esta práctica es necesario **descargar**, en **formato `.py`**, el código de la **práctica anterior** con el nombre **`utils.py`**. Para ello, pulsamos, en la libreta de la primera práctica, **`File > Export .py`**.

Una vez hemos descargado y nombrado correctamente el fichero, lo **añadimos** al espacio de trabajo de la **libreta** de la práctica **actual** a través de **`Attached data > Notebook files > Upload files`** y subimos el fichero `utils.py` descargado en el paso anterior.

Tras esto, debemos **cambiar** el **constructor** de la clase **`Problem`** para que **reciba** directamente el **problema** a resolver, **en lugar de cargarlo desde** un **fichero**. Esto se debe a que los algoritmos metaheurísticos van a tener que resolver, en múltiples ocasiones, este problema. De esta manera nos **evitamos** la **carga computacional extra** que implica **leer** el problema desde un **fichero**. Además, también es necesario **comentar** cualquier línea de **código** que **imprima estadísticas** para evitar salidas largas.

## 1. Introducción

En esta práctica, vamos a **resolver** un **problema** de **optimización combinatoria** mediante **algoritmos metaheurísticos**. En particular, vamos a implementar **algoritmos genéticos** para abordar el **problema** de **ruteo** de **vehículos**. En este, **varios vehículos** con **capacidad limitada** deben **recoger paquetes** en **diferentes ubicaciones** y **trasladarlos** a una **sede central** de recogida.

Además, se **analizará** y **comparará** el **rendimiento** de **diferentes algoritmos genéticos** (mediante la modificación de los pasos correspondientes) en diferentes instancias del problema.

---

## 2. Descripción del problema

El concepto de mapa que vamos a utilizar es similar al de la primera práctica. Este se representa mediante un **grafo**, donde los **nodos** representan **ciudades** y los **enlaces** indican la existencia de una **carretera en ambos sentidos** entre dos ciudades. Además, los **enlaces** tienen un **peso** asociado indicando la **distancia real** entre las dos ciudades. Al mismo tiempo, se proporciona una **sede central**, que se trata de la ciudad donde se deben dejar los paquetes. A su vez, se dispone de una **flota de vehículos** con **capacidad limitada** que deben **recoger** los **paquetes** en las ciudades correspondientes y que **inicialmente** están aparcados en la **sede central**. **En caso de que a la hora de recoger un paquete se supere la capacidad del vehículo correspondiente, este debe volver a la sede central a descargar todos los paquetes, considerando el coste que implicaría volver**.

Un mapa es un problema en particular, pero diferentes paquetes, capacidades de vehículos y ubicación de la sede central pueden dar lugar a diferentes instancias del problema. Por tanto, el **objetivo** en este problema es **recoger todos los paquetes de tal manera que los vehículos recorran la menor distancia posible**.

**Con el objetivo de simplificar la práctica en evaluación continua, se asume que se cuenta con un único vehículo. No obstante, esto podría cambiar para la evaluación no continua.**

---

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica se va a proporcionar un conjunto de mapas, sobre los cuáles se debe resolver el problema de optimización combinatoria correspondiente. Es importante destacar que la **dimensionalidad** del **problema** (número de ciudades, carreteras y paquetes) puede ser **variable**, por lo que los diferentes **algoritmos genéticos** deben ser lo suficientemente **eficientes** para que puedan **resolver** los **problemas** en un **tiempo razonable**.

**Además, algunos escenarios se van a guardar para las entrevistas de prácticas, por lo que el código debe ser lo más general posible para cargarlos de manera rápida y sencilla.**

### 3.1. Entrada

Cada escenario tendrá un fichero `.json` asociado con la siguiente estructura:

```JSON
{
    "map": {
        "cities": [
            {
                "id": id_city_0,
                "name": name_city_0,
                "lat": latitude_city_0,
                "lon": longitude_city_0
            }
        ],
        "roads": [
            {
                "origin": origin_city_id,
                "destination": destination_city_id,
                "distance": road_distance
            }
        ]
    },
    "warehouse": warehouse_city_id,
    "vehicles": [
        {
            "id": id_vehicle_0,
            "capacity": capacity_vehicle_0
        }
    ]
    "parcels": [
        {
            "id": id_parcel_0,
            "city": parcel_city_id,
            "weight": weight_parcel_0
        }
    ]
}
```

Hay cuatro elementos principales en el fichero:

* `map`: Un diccionario con el mapa, cuya descripción es la misma que la de la primera práctica
* `warehouse`: Identificador de la ciudad donde se encuentra la sede central
* `vehicles`: Lista de vehículos disponibles
* `parcels`: Lista de paquetes a recoger

Por su parte, `vehicles` contiene:

* `id`: Identificador del vehículo
* `capacity`: Capacidad máxima del vehículo

Y `parcels`:

* `id`: Identificador del paquete
* `city`: Ciudad donde se encuentra el paquete
* `weight`: Peso del paquete

**Para añadir los ficheros con los problemas al espacio de trabajo se debe usar el mismo procedimiento anterior.**

---

## 4. Plan de trabajo

### 4.1. Formalización del problema

Para resolver cualquier problema de optimización combinatoria en primer lugar hay que definir como vamos a **codificar** las **soluciones** al problema. Si bien es algo que se deja a criterio propio, se plantea la siguiente pregunta, **¿cuál puede ser la mejor representación para una secuencia de paquetes a recoger?**

Se puede comprobar si la respuesta es correcta introduciéndola en la variable `answer` del siguiente fragmento de código:

In [24]:
# Third party
import hashlib
import json 
import random
import numpy as np
import time
import concurrent.futures
import multiprocessing
import threading

In [25]:
check_answer = lambda answer, hashed: "The answer is " + ("" if hashlib.md5(answer).hexdigest() == hashed else "in") + "correct."

In [26]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

hashed = "90d377b31e1ac26d0d10d5612ce33ccc"  # The hashed answer
print(hashed)

check_answer(answer, hashed)

90d377b31e1ac26d0d10d5612ce33ccc


'The answer is incorrect.'

### 4.2. Implementación

A continuación se proporciona la estructura de clases recomendada para resolver el problema en cuestión. Tendréis que completar las siguientes clases de acuerdo con los algoritmos estudiados en teoría. **Debéis incluir en la siguiente celda todas las librerías que vayáis a utilizar para mantener la libreta lo más organizada posible**:

In [27]:
# Importamos la clase utils.py en la carpeta actual
import utils as ut

#### Clase `Individual`

Esta clase proporciona la **codificación** de un **individuo** de la **población**.

Los **métodos obligatorios** que se deben añadir son:

* ``__init__(self, num_genes, generation_type, crossover_type, mutation_type)``: Inicializa el **número** de **genes** del **individuo** y el **tipo** de **operación** de **generación**, **cruce** y **mutación**. Ademas, genera la **solución** que **representa** el **individuo**.
* ``generate(num_genes, generation_type)``: Método estático para **generar** una **solución** del tamaño proporcionado de acuerdo con el tipo de operación de generación.
* ``crossover(self, individual)``: **Cruza** el **individuo actual** con el **individuo** de **entrada** de acuerdo con el tipo de operación de cruce.
* ``mutation(self)``: **Muta** el **individuo** de acuerdo con el tipo de operación de mutación.
* ``evaluate(self, problem)``: **Evalua** el **individuo** usando el **problema** a **resolver**.

Y los **métodos recomendados** son:

* ``__str__(self)``: **Representación** en formato de **cadena** de **caracteres** de un **individuo**. Método útil para depurar una lista de individuos.
* ``__repr__(self)``: **Método** invocado cuando se ejecuta **``print``** sobre el **individuo**. Método útil para depurar un solo individuo.

In [28]:
class Individual:

    # =============================================================================
    # Constructor
    # =============================================================================

    # Creamos un objeto de random para poder utilizarlo en los metodos estaticos
    #random = random.Random(123)

    def __init__(self, num_genes, generation_type="random", crossover_type = None, mutation_type = None):
        self.num_genes = num_genes
        self.generation_type = generation_type
        self.crossover_type = crossover_type
        self.mutation_type = mutation_type

        self.fitness = 0
        
        self.genes = []


    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # Método estático para generar una solución del tamaño proporcionado de acuerdo con el tipo de operación 
    # de generación.
    # A este metodo le pasamos num_genes, generation_type y un objeto de la clase random
    # @staticmethod
    def generate(num_genes, generation_type="random"):
        # Hay diferentes tipos de generación de individuos -> aleatoria, por distribucion, por enfoque, por seleccion...
        
        # Generación aleatoria -> Se generan los genes aleatoriamente

        # Generacion aleatoria para numeros enteros        
        if generation_type == "random":
            # Creamos un individuo vacio
            individual = Individual(num_genes, generation_type)
        
            # Generamos los genes de forma aleatoria sin que se repitan
            individual.genes = random.sample(range(num_genes), num_genes)

            return individual
        

    # Método para realizar el cruce de dos individuos de acuerdo con el tipo de operación de cruce.
    def crossover(self, individual):
        # Tenemos diferentes tipos de operadores de cruce nosotros utilizaremos el que se utiliza para situaciones 
        # reales, el cruce 2PCX (Two Point Center Crossover)

        # Creamos los puntos de corte aleatoriamente, para hacer que siempre se generen dos puntos de corte debemos evitar
        # que los puntos de corte sean iguales y que los puntos de corte sean el primer o ultimo gen y que los puntos de corte
        # es decir que no sean 0 o num_genes - 1 

        cut_points = random.sample(range(1, self.num_genes - 1), 2)

        # Asignamos los puntos de corte segun el orden de menor a mayor
        cut_point_1, cut_point_2 = min(cut_points), max(cut_points)

        list_1 = self.genes[cut_point_1:cut_point_2]
        list_2 = individual.genes[cut_point_1:cut_point_2]

        child_1 = self.genes[:cut_point_1] + [num for num in individual.genes if num in list_1] + self.genes[cut_point_2:]
        individual.genes = individual.genes[:cut_point_1] + [num for num in self.genes if num in list_2] + individual.genes[cut_point_2:]
        self.genes = child_1

        return self, individual


    # def crossover(self, individual):
    #     # Tenemos diferentes tipos de operadores de cruce nosotros utilizaremos el que se utiliza para situaciones 
    #     # reales, el cruce 2PCX (Two Point Center Crossover)

    #     # Creamos los puntos de corte aleatoriamente, para hacer que siempre se generen dos puntos de corte debemos evitar
    #     # que los puntos de corte sean iguales y que los puntos de corte sean el primer o ultimo gen y que los puntos de corte
    #     # es decir que no sean 0 o num_genes - 1 

    #     cut_points = random.sample(range(1, self.num_genes - 1), 2)

    #     # Asignamos los puntos de corte segun el orden de menor a mayor
    #     cut_point_1, cut_point_2 = min(cut_points), max(cut_points)

    #     # Creamos los hijos
    #     child_1 = self.genes[:cut_point_1] + [x for x in individual.genes if x in self.genes[cut_point_1:cut_point_2]] + self.genes[cut_point_2:]
    #     individual.genes = individual.genes[:cut_point_1] + [x for x in self.genes if x in individual.genes[cut_point_1:cut_point_2]] + individual.genes[cut_point_2:]
    #     self.genes = child_1
        
    #     return self, individual


    # Método para realizar la mutación de un individuo de acuerdo con el tipo de operación de mutación.
    def mutation(self):
        
        # Tenemos diferentes mecanismos de mutación para este problema utilizaremos permutaciones, en concreto
        # Intercambio e inserción
        # 1. Intercambio -> Seleccionamos dos genes aleatoriamente y los intercambiamos
        # 2. Inserción -> Seleccionamos un gen aleatoriamente y lo insertamos en una posición aleatoria
        

        # Seleccionamos el tipo de mutación
        mutation_type = random.randint(0, 1)

        mutation_type = 0

        # Intercambio
        if mutation_type == 0:
            # print("\nINTERCAMBIO\n")
            # Seleccionamos dos genes aleatoriamente
            genes = random.sample(range(self.num_genes), 2)
            # Intercambiamos los genes
            self.genes[genes[0]], self.genes[genes[1]] = self.genes[genes[1]], self.genes[genes[0]]
        
        # Inserción
        else:
            # print("\nINSERCION\n")
            # Seleccionamos un gen aleatoriamente
            gene = random.randint(0, self.num_genes - 1)
            # print("GEN: ", gene)
            # Seleccionamos una posición aleatoria
            position = random.randint(0, self.num_genes - 1)
            # print("POSICION: ", position)
            # Insertamos el gen en la posición
            self.genes.insert(position, self.genes.pop(gene))

        return self.genes

        

        
        
    
    def evaluate(self, problem):
        # Evaluamos el individuo
        self.fitness = problem.evaluate(self.genes)
        return self.fitness
        
        
    def evaluate_first_number(self):
        # Evaluamos el individuo en base al primer numero de la lista de genes
        self.fitness = self.genes[0]

        return self.fitness
    

    def hill_climb_individual(self, problem):
        # Creamos un individuo con una lista de genes aleatoria
        individual = Individual(random.sample(range(1, self.num_genes + 1), self.num_genes))
        # Evaluamos el individuo
        individual.evaluate(problem)
        # Evaluamos el individuo actual
        self.evaluate(problem)
        # Si el individuo actual es mejor que el individuo generado
        if self.fitness > individual.fitness:
            # Devolvemos el individuo actual
            return self
        # Si el individuo generado es mejor que el individuo actual
        else:
            # Devolvemos el individuo generado
            return individual

    # =============================================================================
    # Recommended methods
    # =============================================================================
    

    def __str__(self):
        return f' Individual: {self.genes}'

    def __repr__(self):
        return f' Individual: {self.genes}'

    

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [29]:
# TODO: Test here the methods to generate a solution

# Creamos un individuo
individual = Individual.generate(10, "random")

# Imprimimos el individuo
print(individual)




 Individual: [6, 2, 8, 7, 0, 4, 3, 5, 9, 1]


In [30]:
# TODO: Test here the methods to cross individuals

# Creamos dos individuos
individual_1 = Individual.generate(5, "random")
individual_2 = Individual.generate(5, "random")

# Imprimimos los individuos
print("Individuo 1\n", individual_1)
print("Individuo 2\n", individual_2)

# Realizamos el cruce
child_1, child_2 = individual_1.crossover(individual_2)

# Imprimimos los hijos
print("Hijo 1", child_1)
print("Hijo 2", child_2)


Individuo 1
  Individual: [2, 4, 1, 0, 3]
Individuo 2
  Individual: [2, 0, 3, 1, 4]
Hijo 1  Individual: [2, 1, 4, 0, 3]
Hijo 2  Individual: [2, 0, 3, 1, 4]


In [31]:
# TODO: Test here the methods to mutate an individual

# Creamos un individuo
individual = Individual.generate(5, "random")

# Imprimimos el individuo
print("Individuo\n", individual)

# Realizamos la mutación
individual.mutation()

# Imprimimos el individuo mutado
print("Individuo mutado\n", individual)



Individuo
  Individual: [0, 3, 4, 2, 1]
Individuo mutado
  Individual: [0, 3, 4, 1, 2]


#### Clase `Genetic`

Esta clase implementa un **esquema** básico de **algoritmo genético**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)``: Inicializa el **tamaño** de la **población**, el **tipo** de **operación** de **selección**, **cruce**, y **mutación**, así como la **probabilidad** de aplicar las operaciones de **cruce** y **mutación**. Además, también inicializa el **número** de **mejores soluciones** de la **población actual** que se **mantienen** en la **siguiente población** y una **semilla** para garantizar que los **experimentos** son **reproducibles**. **Nótese que puede ser necesario añadir más argumentos si así se requiere**.
* ``def __call__(self, problem)``: **Método** que se **ejecuta** cuando se llama a un **objeto** de la **clase como** si fuese una **función**. En este **método** se debe **implementar** el **esquema básico** de un **algoritmo genético** que se encargue de ejecutar los pasos correspondientes. 
* ``def generate_population(self, problem)``: **Genera** la **población inicial** de acuerdo con el **problema** a resolver.
* ``def select_population(self, population, scores)``: **Selecciona** los **padres** a utilizar para la operación de cruce.
* ``def crossover(self, population)``: **Cruza pares** de **padres** teniendo en cuenta la probabilidad de cruce.
* ``def mutation(self, population)``: **Muta** los **individuos cruzados** teniendo en cuenta la probabilidad de mutación.
* ``def evaluate(self, population, problem)``: **Evalua** los **nuevos individuos** de acuerdo con el problema a resolver.
* ``def combine(self, population)``: **Forma** la **nueva generación** de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.

In [32]:



class Genetic:

    # =============================================================================
    # Constructor
    # =============================================================================

    def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state, parrallel=True, hill_climbing=False):
        
        # Tamaño de la población
        self.population_size = population_size

        # Numero de generaciones
        self.num_generations = num_generations

        # Tipo de selección
        self.selection_type = selection_type

        # Tipo de cruce
        self.crossover_type = crossover_type

        # Probabilidad de cruce
        self.crossover_probability = crossover_probability

        # Tipo de mutación
        self.mutation_type = mutation_type

        # Probabilidad de mutación
        self.mutation_probability = mutation_probability

        # Lista de individuos
        self.population = []

        self.population_set = set()

        # Si queremos mantener el elitismo, es decir, el mejor individuo de la población anterior 
        self.keep_elitism = keep_elitism

        # Hill Climbing
        self.hill_climbing = hill_climbing
        
        # Semilla aleatoria, la semilla aleatoria es un número que se utiliza para inicializar un generador de números pseudoaleatorios.
        # Esto nos ayuda a obtener los mismos resultados cada vez que ejecutamos el algoritmo
        random.seed(random_state)

        # Si queremos que el algoritmo ejecute las evaluaciones en paralelo
        self.parrallel = parrallel


    def __call__(self, problem):
        # self.problem = problem
        # Call es el encargado de llamar a los métodos de la clase Genetic y de esta forma poder ejecutar el algoritmo genético
        # Para ello seguiremos los siguientes pasos:

        # 1. Inicializar la población: utilizar el método generate_population para generar una población de individuos de tamaño 
        # population_size, utilizando el problema especificado en el argumento problem.

        # 2. Evaluar la población: utilizar el método evaluate para evaluar a cada individuo de la población y obtener sus puntuaciones.

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations):

        # 4. Seleccionar individuos de la población: utilizar el método select_population para seleccionar un conjunto de individuos de la 
        # población utilizando el método de selección especificado en el argumento selection_type.

        # 5. Aplicar crossover: utilizar el método crossover para aplicar el tipo de crossover especificado en el argumento crossover_type a 
        # los individuos seleccionados, con una probabilidad especificada en el argumento crossover_probability.

        # 6. Aplicar mutación: utilizar el método mutation para aplicar el tipo de mutación especificado en el argumento mutation_type a los 
        # individuos resultantes del crossover, con una probabilidad especificada en el argumento mutation_probability.

        # 7. Evaluar la nueva población: utilizar el método evaluate para evaluar a cada individuo de la nueva población y obtener sus puntuaciones.

        # 8. Combinar la población: utilizar el método combine para combinar la población actual con la nueva población generada, 
        # teniendo en cuenta el argumento keep_elitism para determinar si se deben conservar los individuos más aptos de la población actual.
                

        # 1. Inicializar la población
        if self.hill_climbing:
            # Generamos la población utilizando hill climbing
            self.generate_population_hill_climb(problem)
        else:
            # Generamos la población normal
            self.generate_population(problem)

        # 2. Evaluar la población, al llamar a la función evaluate se le asigna a la variable fitness en la clase Individual la puntuación
        if self.parrallel:
            # Si queremos que el algoritmo se ejecute en paralelo
            self.evaluate_parallel(self.population, problem)
        else:
            self.evaluate(self.population, problem)

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations)
        for _ in range(self.num_generations):
                
                # 4. Seleccionar individuos de la población
                selected_population = self.select_population(self.population)
    
                # 5. Aplicar crossover
                crossed_population = self.crossover(selected_population)
    
                # 6. Aplicar mutación
                mutated_population = self.mutation(crossed_population)
    
                # 7. Evaluar la nueva población
                if self.parrallel:
                    # Si queremos que el algoritmo se ejecute en paralelo
                    self.evaluate_parallel(mutated_population, problem)
                else:
                    self.evaluate(mutated_population, problem)
    
                # 8. Combinar la población
                self.population = self.combine(mutated_population)

        # Devolvemos la mejor solución, es decir la que tenga la puntuación mas baja
        return min(self.population, key=lambda x: x.fitness)
        # return self.population





    # =============================================================================
    # Mandatory methods
    # =============================================================================

    def generate_population(self, problem):
        
        # Ahora creamos la poblacion para el problema en si, es decir, CVRP
        # Si problem es de tipo int es que es un problema de prueba 
        # population = []

        if isinstance(problem, int):
            self.population = [Individual.generate(problem, "random") for _ in range(self.population_size)]
        else:
            self.population = [Individual.generate(len(problem.parcels_dict), "random") for _ in range(self.population_size)]
 
        
    def generate_population_hill_climb(self, problem):
        # Para generar una poblacion inicial utilizando hill climb, llamamos a este metodo. 
        # Para esto debemos seguir los siguientes pasos:
        
        # 1. Utilice la búsqueda por ascenso de colinas para encontrar el mejor individuo en una población inicial aleatoria.

        # Utilice este mejor individuo como un punto de inicio para la búsqueda por ascenso de colinas en varias iteraciones adicionales, generando un conjunto de individuos óptimos.
        
        # Utilice estos individuos óptimos como la población inicial para el algoritmo genético.

        # Creamos una poblacion inicial aleatoria
        self.generate_population(problem)

        # Evaluamos la poblacion inicial
        self.evaluate(self.population, problem)

        # Se elige el mejor individuo de la poblacion
        best_individual = min(self.population, key=lambda x: x.fitness)

        


        # Definimos un numero de iteraciones para hill climb
        num_iterations = 100

        # Creamos un conjunto para guardar los individuos optimos
        optimal_individuals = set()

        # Añadimos el mejor individuo a la poblacion
        optimal_individuals.add(best_individual)

        # Hacemos hill climb hasta que el numero del conjunto de individuos optimos sea igual al numero de individuos de la poblacion (population_size)
        # while len(optimal_individuals) < self.population_size:
        for _ in range(num_iterations):
            # Aplicamos hill climb al mejor individuo
            best_individual = best_individual.hill_climb_individual(problem)

            # Guardamos el mejor individuo en el conjunto de individuos optimos
            optimal_individuals.add(best_individual)
        

        # Convertimos el conjunto de individuos optimos en una lista
        population = list(optimal_individuals)

        # Comprobamos si el tamaño de population == population_size
        if len(population) != self.population_size:
            # Si no es asi, añadimos los mejores individuos de self.population hasta que population tenga tamaño population_size
            population += sorted(self.population, key=lambda x: x.fitness)[:self.population_size - len(population)]

        assert len(population) == self.population_size
        assert len(self.population) == len(population)

        # Asignamos la poblacion creada con hill climb a la poblacion del algoritmo genetico, con limite tamaño de la poblacion
        self.population = population[:self.population_size]
        

        

        

    def hill_climb(self, population, problem):
        # Aplicamos hill climb a cada individuo de la poblacion
        for individual in population:
            # Aplicamos hill climb a cada individuo de la poblacion
            individual = self.hill_climb_individual(individual, problem)

        # Devolvemos el mejor individuo
        return min(population, key=lambda x: x.fitness)

    @staticmethod
    def select_tournament(population):
        # Seleccion por torneo 
        # Seleccionamos dos individuos aleatorios de la población y nos quedamos con el mejor 

        # Seleccionamos dos individuos aleatorios de la población
        individuals = random.sample(population, 2)

        # Nos quedamos con el mejor
        best_individual = min(individuals, key=lambda x: x.fitness)

        return best_individual


    def select_population(self, population, num_individuos_selected=None):
        # Selecciona un conjunto de individuos de la población utilizando el método de selección especificado en el argumento selection_type.
        selected_population = set()

        # El numero de individuos seleccionados debe ser menor o igual que el tamaño de la población, mayor que 0 y par
        if num_individuos_selected is None:
            # Si no se especifica el numero de individuos seleccionados, se selecciona la mitad de la población
            num_individuos_selected = self.population_size // 2
            
    
        # Una vez especificado el numero de individuos seleccionados, comprobamos que sea menor o igual que el tamaño de la población, mayor que 0 y par
        if num_individuos_selected > self.population_size or num_individuos_selected <= 0:
            raise ValueError("El numero de individuos seleccionados debe ser menor o igual que el tamaño de la población, mayor que 0 y par")
        if num_individuos_selected % 2 != 0:
            # Si el numero de individuos seleccionados no es par, lo hacemos par
            # Para ello podemos restarle 1 o sumarle 1 sumaremos siempre 1 a menos que num_individuos_selected sea igual que self.population_size
            if num_individuos_selected == self.population_size or num_individuos_selected == self.population_size - 1:
                num_individuos_selected -= 1
            else:
                num_individuos_selected += 1



        if self.selection_type == "torneo":
            # Seleccion por torneo
            for _ in range(num_individuos_selected):
                # Añadimos el individuo seleccionado por torneo a la población
                selected_population.add(self.select_tournament(population))


        elif self.selection_type == "roulette":
            # Seleccion por ruleta
            pass


        elif self.selection_type == "ranking":
            # Seleccion por ranking
            pass


        elif self.selection_type == "random":
            # Seleccion aleatoria
            pass
        
        else:
            raise Exception("Invalid selection type")

        # Convertimos la población seleccionada en una lista
        selected_population = list(selected_population)

        return selected_population


    def crossover(self, selected_population):
        # Cruza pares de padres teniendo en cuenta la probabilidad de cruce.
        # Para la seleccion de los padres utilizamos el metodo select_population
        # Para el cruce utilizamos el metodo crossover de la clase Individual

        # El cruce se realiza teniendo en cuenta la probabilidad de cruce self.crossover_probability
        # Si la probabilidad de cruce es mayor que el numero aleatorio generado entonces se realiza el cruce
        # Si la probabilidad de cruce es menor que el numero aleatorio generado entonces no se realiza el cruce
        
        # Creamos un numero aleatorio entre 0 y 1 para realizar la probabilidad de cruce
        random_number = random.random()
        if random_number < self.crossover_probability:
            # Comprobamos la longitud de la población seleccionada
            # Si la longitud es par realizamos el cruce de todos los seleccionados en pares
            # Si la longitud es impar realizamos el cruce de todos los seleccionados en pares menos el ultimo
            # print("Cruce")

            if len(selected_population) % 2 == 0:
                # Creamos una lista con los padres en pares
                parents = [selected_population[i:i+2] for i in range(0, len(selected_population), 2)]
            else:
                # Creamos una lista con los padres en pares menos el ultimo
                parents = [selected_population[i:i+2] for i in range(0, len(selected_population) - 1, 2)]

            # Creamos una lista para los hijos
            childs = []

            for parent1, parent2 in parents:
                # Realizamos el cruce de los padres
                child1, child2 = parent1.crossover(parent2)

                # Añadimos los hijos a la lista de hijos
                childs += [child1, child2]
            
        else:
            # print("Cruce no realizado")
            # Si no se realiza el cruce, devolvemos la población seleccionada sin cruzar
            return selected_population
            

        return childs




    def mutation(self, population):
        # Añadimos una probabilidad de mutación
        for individual in population:
            if random.random() < self.mutation_probability:
                individual.mutation()
        return population


    # def evaluate(self, population, problem):
    #     # Evaluamos los nuevos individuos de acuerdo con el problema a resolver, devolviendo una lista con las puntuaciones de cada individuo
    #     scores = [problem.evaluate(individual) for individual in population]

    #     return scores

    def evaluate(self, population, problem):
        # Evaluamos los individuos de acuerdo con el problema a resolver
        for individual in population:
            if individual.fitness == 0:
                individual.evaluate(problem)



    # # Para mejorar la eficiencia del metodo evaluate, lo hacemos de forma paralela
    # def evaluate_parallel(self, population, problem):
    #     # Evaluamos los individuos de acuerdo con el problema a resolver
    #     with concurrent.futures.ProcessPoolExecutor() as executor:
    #         executor.map(lambda individual: individual.evaluate(problem), population)


    # POOL MULTIPROCCESING

    # def evaluate_individual(individual, problem):
    #     individual.evaluate(problem)

    # def evaluate_parallel(self, population, problem):
        
    #     # Creamos un pool de procesos con 4 procesos
    #     with Pool(4) as p:
    #         # Evaluamos los individuos de acuerdo con el problema a resolver
    #         for individual in population:
    #             if individual.fitness == 0:
    #                 p.apply_async(self.evaluate_individual, args=(individual, problem))

    #         # Esperamos a que todos los procesos terminen
    #         p.close()
    #         p.join()


    # THREADING
    def evaluate_parallel(self, population, problem):

        def evaluate_individual(individual, problem):
            individual.evaluate(problem)

        # Creamos una lista de hilos
        threads = []

        # Evaluamos los individuos de acuerdo con el problema a resolver
        for individual in population:
            if individual.fitness == 0:
                # Creamos un hilo para evaluar el individuo y lo añadimos a la lista de hilos
                thread = threading.Thread(target=evaluate_individual, args=(individual, problem))
                threads.append(thread)

        # Lanzamos todos los hilos
        for thread in threads:
            thread.start()

        # Esperamos a que todos los hilos terminen
        for thread in threads:
            thread.join()



    
            # def add_individuals_parallel(new_population_part, option, new_population_2):
                
            #     # Opcion 1: Añadimos todos los elementos de la nueva poblacion que no esten en la poblacion actual
            #     if option == 1:
            #         for individual in new_population_part:
            #             if individual not in self.population_set:
            #                 new_population_2.append(individual)
            #                 self.population_set.add(individual)
            #                 # Numero de elementos añadidos a la nueva poblacion
            #                 num_added_new_p += 1

            #     # Opcion 2: Añadimos los elementos de la nueva poblacion (50%) que no esten en la poblacion actual hasta que se llene la nueva poblacion
            #     else:
            #         # new_population_2.extend(new_population[:population_size_50])
            #         for individual in new_population_part:
            #             if individual not in population_set:
            #                 new_population_2.append(individual)
            #                 population_set.add(individual)
            #                 # Numero de elementos añadidos a la nueva poblacion
            #                 num_added_new_p += 1
            #             if num_added_new_p == population_size_50:
            #                 # Paramos si se ha llegado al 50% de la poblacion actual
            #                 break


    def combine(self, new_population):
        # Si se quiere mantener el elitismo
        if self.keep_elitism:
            # Ordenamos las poblaciones de menor a mayor fitness
            new_population.sort(key=lambda x: x.fitness)
            self.population.sort(key=lambda x: x.fitness)

            # Creamos una variable population size para utilizarla como auxiliar 
            population_size = self.population_size
            new_population_size = len(new_population)

            # Mezclamos el 50% de la poblacion actual con el 50% de la nueva poblacion (los mejores con los mejores de cada poblacion)

            # Creamos una nueva poblacion
            new_population_2 = []
            # Creamos una variable para guardar el tamaño de la poblacion actual dividido entre 2
            population_size_50 = population_size // 2

            # Hacemos un conjunto de los elementos de la poblacion actual y la nueva poblacion
            population_set = set(self.population)

            

            # Si la nueva poblacion es menor que el 50% de la poblacion actual, añadimos todos los elementos de la nueva poblacion que no esten en la poblacion actual
            if new_population_size < population_size_50:
                option = 1
            else:
                option = 2
            
            def add_individuals(new_population_part, option):

                num_added_new_p = 0
                # Opcion 1: Añadimos todos los elementos de la nueva poblacion que no esten en la poblacion actual
                if option == 1:
                    for individual in new_population_part:
                        if individual not in population_set:
                            new_population_2.append(individual)
                            population_set.add(individual)
                            # Numero de elementos añadidos a la nueva poblacion
                            num_added_new_p += 1
                    return num_added_new_p

                # Opcion 2: Añadimos los elementos de la nueva poblacion (50%) que no esten en la poblacion actual hasta que se llene la nueva poblacion
                else:
                    # new_population_2.extend(new_population[:population_size_50])
                    for individual in new_population_part:
                        if individual not in population_set:
                            new_population_2.append(individual)
                            population_set.add(individual)
                            # Numero de elementos añadidos a la nueva poblacion
                            num_added_new_p += 1
                        if num_added_new_p == population_size_50:
                            # Paramos si se ha llegado al 50% de la poblacion actual
                            return num_added_new_p
                    return num_added_new_p


            ##################################### COMBINE PARALELO
            # num_processes = 4
            # Creamos una lista de listas con los elementos de la nueva poblacion divididos en 4 partes
            # new_population_parts = [new_population[i:i + len(new_population)//num_processes] for i in range(0, len(new_population), len(new_population)//num_processes)]

            # Creamos una lista de procesos
            # processes = []
            # Creamos un proceso por cada parte de la nueva poblacion
            # for i in range(num_processes):
            #     # Creamos un proceso
            #     p = Process(target=add_individuals, args=(new_population_parts[i], option))
            #     # Añadimos el proceso a la lista de procesos
            #     processes.append(p)
            #     # Iniciamos el proceso
            #     p.start()

            # # Esperamos a que terminen todos los procesos
            # for p in processes:
            #     p.join()

            ########################################## COMBINE NORMAL
            num_added_new_p = add_individuals(new_population, option)

            # Añadimos la poblacion actual a la nueva poblacion
            # Si no se han añadido todos los elementos de la nueva poblacion, añadimos los faltantes de la poblacion actual
            if num_added_new_p < population_size_50:
                # Añadimos a la nueva poblacion 2 los elementos faltantes de la poblacion actual
                new_population_2.extend(self.population[:population_size - num_added_new_p])

            # La nueva poblacion 2 debe tener el tamaño de la poblacion actual
            assert len(new_population_2) == population_size

        # Si no se quiere mantener el elitismo
        else:
            # Ordenamos la nueva poblacion de menor a mayor fitness
            new_population.sort(key=lambda x: x.fitness)
            self.population.sort(key=lambda x: x.fitness)

            # Debemos devolver una poblacion de tamaño igual a la poblacion actual por tanto:
            # Añadimos todos los elementos de la nueva poblacion
            new_population_2 = new_population
            # Añadimos todos los elementos faltantes de la poblacion actual
            new_population_2.extend(self.population[:self.population_size - len(new_population)])

            # Aqui la nueva poblacion 2 debe tener el tamaño de la poblacion actual
            assert len(new_population_2) == self.population_size
            


        # Ordenamos la nueva poblacion de menor a mayor fitness
        new_population_2.sort(key=lambda x: x.fitness)
        
        return new_population_2[:self.population_size]

            


    


**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [33]:
# TODO: Test here the methods to generate a population
# Probamos el metodo para generar una población inicial de Genetic 
# self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)
# A genetic le pasamos estos parametros
gn = Genetic(5, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

# Creamos una población inicial
gn.generate_population(65)

# Probamos a generar la poblacion con hill climbing
# gn.generate_population_hill_climb(65)

# Imprimimos la población inicial
print('Población inicial')
for individual in gn.population:
    print(individual.genes)





Población inicial
[17, 8, 16, 7, 31, 48, 28, 30, 41, 24, 50, 13, 6, 60, 1, 55, 27, 38, 0, 44, 58, 64, 14, 37, 52, 20, 54, 47, 56, 34, 46, 49, 53, 59, 23, 35, 62, 61, 33, 42, 15, 43, 51, 11, 22, 39, 25, 9, 29, 32, 26, 63, 10, 36, 2, 4, 21, 5, 19, 40, 57, 3, 12, 45, 18]
[36, 63, 54, 60, 32, 25, 37, 62, 2, 30, 15, 47, 51, 59, 26, 42, 11, 23, 35, 44, 43, 53, 5, 28, 61, 6, 10, 33, 52, 45, 31, 1, 55, 56, 9, 22, 27, 19, 18, 50, 12, 20, 49, 29, 16, 7, 0, 39, 41, 24, 8, 21, 40, 13, 46, 4, 14, 64, 3, 48, 57, 58, 34, 38, 17]
[7, 61, 55, 23, 36, 35, 12, 32, 26, 31, 52, 22, 56, 53, 0, 34, 49, 39, 47, 21, 29, 38, 1, 14, 40, 11, 59, 37, 46, 5, 16, 2, 4, 43, 27, 50, 41, 44, 24, 54, 8, 64, 25, 3, 19, 62, 60, 9, 33, 20, 48, 57, 51, 28, 10, 13, 17, 63, 15, 18, 45, 58, 30, 6, 42]
[49, 43, 26, 50, 12, 16, 6, 59, 46, 32, 13, 38, 27, 1, 14, 51, 25, 9, 2, 10, 28, 55, 52, 34, 61, 33, 44, 40, 39, 64, 48, 20, 42, 3, 23, 47, 4, 58, 35, 29, 56, 24, 41, 22, 5, 54, 18, 8, 53, 0, 17, 19, 15, 21, 31, 11, 7, 30, 63, 36

In [34]:
# TODO: Test here the methods to select individuals
# Probamos los metodos de seleccion de individuos 
# Para probar los metodos de seleccion de individuos, vamos a crear una poblacion de 10 individuos y vamos a seleccionar 5 individuos de la poblacion de 10 individuos

# # Probamos el metodo para seleccionar los padres
# parents = gn.select_population(gn.population)

# # Imprimimos los padres seleccionados
# print('Padres seleccionados')
# for individual in parents:
#     print(individual)

# print(type(parents))
# print(type(parents[0]))


In [35]:
# print(parents)

# # Probramos el metodo para cruzar y mutar los padres
# childs = gn.crossover(parents)

# # print(childs)
# # Imprimimos los hijos
# print('Hijos cruzados')
# for individual in childs:
#     print(individual)
#     # print(type(individual))

# # Probamos el metodo para mutar los hijos
# childs = gn.mutation(childs)

# # Imprimimos los hijos mutados
# print('Hijos mutados')
# for individual in childs:
#     print(individual)
#     # print(type(individual))

    
# print(type(childs))
# print(type(childs[0]))
# print(type(gn.population))


In [36]:
# PRUEBAS INDIVIDUALES DE COMBINE DEBAJO DE CVRP

In [37]:
# Probamos el metodo __call__ de la clase Genetic encargado de ejecutar el algoritmo genetico completo
# gn = Genetic(10, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1)

# Call crea la poblacion y hace los pasos necesarios para ejecutar el algoritmo genetico
# Le pasamos el problema que queremos resolver



#### Clase `CVRP`

Esta clase representa el problema en cuestión, esto es, el **problema** de **ruteo** de **vehículos**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, filename, algorithm)``: Inicializa el **problema** en cuestión y el **algoritmo** a usar para resolverlo. A su vez, se debe crear un **diccionario** que contenga como **clave** un **identificador** de **paquete** y como **valor** una **tupla** con la **ciudad** donde se encuentra dicho paquete y su **peso**.
* ``def __call__(self)``: **Resuelve** el **problema** en cuestión.
* ``def evaluate(self, solution)``: **Evalua** una **solución** para el **problema** en cuestión, **teniendo en cuenta** las **restricciones correspondientes**.
* ``def search(self, departure, goal)``: **Resuelve** un **problema** de **búsqueda** de **caminos** dada las **ciudades** de **salida** y **meta**.

**Nótese que se puede crear una estructura de datos para agilizar el proceso de búsqueda de caminos requerido por el algoritmo ¿cuál puede ser?**

In [38]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

encoded = "0fea6a13c52b4d4725368f24b045ca84"  # The hashed answer
print(encoded)

check_answer(answer, encoded)

0fea6a13c52b4d4725368f24b045ca84


'The answer is incorrect.'

In [39]:
class CVRP:

    # =============================================================================
    # Constructor
    # =============================================================================
    
    # Sobreescribir el método __init__ de la clase problem ubicada en utils.py




# No necesitamos la primera funcion de carga de archivo, la ciudad objetivo ni ciudad inicial
    def __init__(self, filename="Spain/example.json", algorithm=None):

        self.filename = filename

        with open(filename, 'r', encoding='utf8') as file:
            problem = json.load(file)
        


        self.problema = ut.Problem(problem)

        self.genetic_algorithm = algorithm

        
        # Para el problema tenemos 4 elementos principales (JSON):
        # map -> mapa del problema (Práctica 1)
        # warehouses -> almacenes del problema (En este caso solo hay 1)
        # vehicles -> lista de vehículos disponibles
            # id -> identificador del vehículo
            # capacity -> capacidad del vehículo
        # parcels -> lista de paquetes a recoger
            # id -> identificador del paquete
            # city -> ciudad donde se encuentra el paquete 
            # weight -> peso del paquete
        
        # Warehouse -> Ciudad en la que se encuentra el almacén (en el problema solo habra 1 almacén) 
        # Elegimos el primer elemento de la lista de almacenes
        self.warehouse = problem['warehouse']

        # Vehicles -> Lista de vehículos disponibles (lista de diccionarios) (en los problemas solo habra 1 vehiculo)
        self.vehicles = problem['vehicles']

        # Parcels -> Lista de paquetes a recoger
        self.parcels = problem['parcels']

        # Creamos un diccionario para almacenar los paquetes, guardamos el id del paquete como clave y una tupla con la ciudad y el peso como valor
        self.parcels_dict = {parcel['id']:(parcel['city'], parcel['weight']) for parcel in self.parcels}


        # Numero de individuos evaluados reales
        self.real_evaluated_items = 0

        # Numero de individuos evaluados totales
        self.total_evaluated_items = 0

        # Diccionario para no explorar soluciones ya exploradas
        self.explored_solutions = {}

        # Diccionario de busquedas realizadas entre ciudades
        self.searches = {}

        # Cantidad de llamadas al metodo search reales 
        self.real_search_calls = 0

        # Cantidad de "llamadas" al metodo search totales
        self.total_search_calls = 0

        # Diccionario de estadisticas
        self.stats = {}

        
        # Matriz de costes entre ciudades
        
        


    # Resuelve el problema en cuestion
    def __call__(self):

        init_time = time.perf_counter()
        # Llamamos al algoritmo genetico para resolver el problema del CVRP y guardamos la solucion en la variable solution
        solution = self.genetic_algorithm(self)

        # population = self.genetic_algorithm(self)


        # Calculamos el tiempo que ha tardado en resolver el problema
        time_elapsed = time.perf_counter() - init_time


        # Guardamos las estadisticas en el diccionario de estadisticas
        self.stats['time'] = time_elapsed
        # Añadimos a la clave 'searches' el numero de llamadas al metodo search reales con clave 'real' y el numero de llamadas totales con clave 'total'
        self.stats['searches'] = {'real': self.real_search_calls, 'total': self.total_search_calls}
        # Añadimos a la clave 'population' el numero de individuos reales y el numero de individuos totales
        self.stats['population'] = {'real': self.real_evaluated_items, 'total': self.total_evaluated_items}

        # Ordenamos el diccionario de soluciones exploradas segun su fitness
        self.explored_solutions = dict(sorted(self.explored_solutions.items(), key=lambda item: item[1]))

        # Mostramos todos los caminos explorados 
        # print("explore: ", self.explored_solutions)
        # print("Explore len: ", len(self.explored_solutions))

        # Mostramos el que menor fitness tenga
        # print("explore_min: ", min(self.explored_solutions.values()))
        # Hacemos los print anteriores en uno solo
        print("\nProblem: ", self.filename, "\nSolution: ", solution, "\nStats: ", self.stats, "\nScore: ", solution.fitness)

        # Devolvemos el individuo con el menor fitness (explored_solutions es un diccionario ordenado por fitness) elemento 0 de la lista de claves
        return solution

        
        

    # =============================================================================
    # Mandatory methods
    # =============================================================================


    def hill_climb(self, solution):
        # Algoritmo de busqueda local hill climb, utilizado para generar la poblacion inicial del algoritmo genetico
        
        pass
        

    # Funcion para resolver el problema del CVRP
    def solve(self, solution):
        # Resuelve el problema del CVRP
        # Recibe una solucion y devuelve una lista de rutas (lista de listas)
        pass
        

    def evaluate(self, solution):
        # Evalua una solucion para el problema del CVRP
        # El algoritmo genetico nos genera un individuo, una lista aleatoria con el orden de las ciudades a visitar

        # Cosas a tener en cuenta:
            # 1.1 Peso del paquete -> El peso del paquete debe ser menor que la capacidad del vehiculo, si no es asi, la solucion no es valida
            # 1.2 Peso del paquete -> Debemos comprobar antes de ir a la siguiente ciudad si el peso del paquete + el peso de los paquetes en el vehiculo 
            # supera la capacidad del vehiculo, si es asi, debemos volver al almacén para dejar los paquetes e ir a la siguiente ciudad
            # 2. Una solución sera mejor cuando la distancia total recorrida es la menor posible

        # Para resolver el problema del CVRP debemos recorrer la lista de ciudades que nos da el algoritmo genetico y comprobar si la solucion es valida
        # Si la solucion es valida, debemos calcular la distancia total recorrida y devolverla como fitness de la solucion
        
        # Inicializamos variables necesarias

        fitness = 0

        # Variable para almacenar el coste individual de una busqueda
        search_cost = 0

        # Convertimos la solucion en una tupla para poder usarla como clave en el diccionario de soluciones exploradas
        tuple_solution = tuple(solution)

        # Comprobamos si el individuo ha sido evaluado previamente
        if tuple_solution in self.explored_solutions: 
            # Si ya ha sido evaluado, obtenemos el coste del diccionario y lo asignamos al fitness del individuo
            fitness = self.explored_solutions[tuple_solution]
            self.total_evaluated_items += 1

        else:
            # Si no ha sido evaluado, lo evaluamos
            # Creamos una variable para almacenar el peso total de los paquetes que llevamos en el vehiculo
            total_weight = 0

            # Creamos una lista para almacenar las rutas (orden a recoger del id de los paquetes)
            # routes = []

            # Creamos una lista para almacenar los paquetes que llevamos en el vehiculo
            # parcels = []

            # Creamos una variable para almacenar la ciudad en la se encuentra el vehiculo (warehouse)
            current_city = self.warehouse

            # Tenemos una lista de ciudades (solution) la cual debemos recorrer
            if solution is None:
                raise ValueError("Solution is None")
            else:

                
                # Recorremos la lista de paquetes
                for next_parcel in tuple_solution:
                    
                    # Siguiente ciudad
                    next_city = self.parcels_dict[next_parcel][0]

                    # Guardamos el peso del paquete de la siguiente ciudad + el peso total de los paquetes en el vehiculo
                    total_weight += self.parcels_dict[next_parcel][1]

                    # Comprobamos si el peso del paquete total supera la capacidad del vehiculo
                    if total_weight > self.vehicles[0]['capacity']:
                        # Si es asi, debemos volver al almacén para dejar los paquetes e ir a la siguiente ciudad

                        # Comprobamos si la ciudad del paquete actual y el almacen han sido calculadas previamente
                        if (current_city, self.warehouse) in self.searches:
                            # Si es asi, obtenemos el coste del diccionario y lo sumamos al fitness
                            fitness += self.searches[(current_city, self.warehouse)]
                            self.total_search_calls += 1
                        else: 
                            # Calculamos la distancia entre la ciudad en la que nos encontramos y la siguiente ciudad y la sumamos al fitness
                            search_cost = self.search(current_city, self.warehouse)

                            # Añadimos el coste al fitness
                            fitness += search_cost

                            # Añadimos el coste al diccionario de costes
                            self.searches[(current_city, self.warehouse)] = search_cost
                            self.real_search_calls += 1

                        # Añadimos el id de los paquetes que llevamos en el vehiculo a la lista de rutas
                        # routes.append(parcels)
                        # Vaciamos la lista de paquetes que llevamos en el vehiculo
                        # parcels = []

                        # Actualizamos la ciudad en la que nos encontramos
                        current_city = self.warehouse
                        # Actualizamos el peso total de los paquetes que llevamos en el vehiculo
                        total_weight = 0

                    # Comprobamos si la ciudad actual y la siguiente ciudad han sido calculadas previamente
                    if (current_city, next_city) in self.searches:
                        # Si es asi, obtenemos el coste del diccionario y lo sumamos al fitness
                        fitness += self.searches[(current_city, next_city)]
                        self.total_search_calls += 1
                    else: 
                        # Calculamos la distancia entre la ciudad en la que nos encontramos y la siguiente ciudad y la sumamos al fitness
                        search_cost = self.search(current_city, next_city)
                        fitness += search_cost
                        # Añadimos el coste al diccionario de costes
                        self.searches[(current_city, next_city)] = search_cost
                        self.real_search_calls += 1


                    # Viajamos a la siguiente ciudad
                    
                    # Añadimos el id del paquete de la siguiente ciudad a la lista de paquetes que llevamos en el vehiculo
                    # parcels.append(next_parcel)
                    # Actualizamos la ciudad actual
                    current_city = self.parcels_dict[next_parcel][0]

                # Una vez recorridas todas las ciudades, debemos volver al almacén para dejar los paquetes
                # Calculamos la distancia entre la ciudad en la que nos encontramos y el almacén, la sumamos al fitness
                # fitness += self.search(current_parcel, self.warehouse)
                # Añadimos el id de los paquetes que llevamos en el vehiculo a la lista de rutas
                # routes.append(parcels)

            # Añadimos el fitness al diccionario de soluciones exploradas, con la solucion como clave 
            self.explored_solutions[tuple_solution] = fitness
            self.total_evaluated_items += 1
            self.real_evaluated_items += 1

        # Devolvemos el fitness de la solucion
        return fitness


    
    def search(self, departure, goal):
        # Asignamos a la variable departure la ciudad de origen, es decir, en la que nos encontramos
        self.problema.departure = departure
        # Asignamos a la variable goal la ciudad de destino, es decir, en la que queremos llegar
        self.problema.goal = goal

        # Llamamos al metodo search en concreto el metodo A* para resolver el problema
        # En el archivo utils he cambiado el return de do_search llamando a la funcion que me calcula el coste
        # del camino que recorre el algoritmo, es decir, la distancia entre la ciudad de origen y la ciudad de destino
        return ut.AStar(self.problema).do_search()


PRUEBAS COMBINE (GENETIC)

In [41]:
# Probamos combine de genetic
# Evaluamos los nuevos individuos
# Tenemos 2 individuos nuevos (childs) y 5 individuos de la poblacion actual (gn.population)
problem = "more_cases-low_dimension\problems\example.json"
geneticAlgorithm = Genetic(10, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

cvrp = CVRP(problem, geneticAlgorithm)

# geneticAlgorithm.generate_population(cvrp)
# Creamos la poblacion con hill climbing
geneticAlgorithm.generate_population_hill_climb(cvrp)

print(geneticAlgorithm.population)

# Mostramos el fitness de los individuos de la poblacion
for i in range(len(geneticAlgorithm.population)):
    print (geneticAlgorithm.population[i].fitness)

# # Evaluamos los individuos de la poblacion
# geneticAlgorithm.evaluate(geneticAlgorithm.population, cvrp)

# for i in range(len(geneticAlgorithm.population)):
#     print (geneticAlgorithm.population[i].fitness)

# # Creamos una nueva poblacion y la combinamos con la poblacion actual
# geneticAlgorithm2 = Genetic(10, 10, "torneo", 'crossover', 0.8, 'mutation', 0.8, True, 5, False)
# geneticAlgorithm2.generate_population(cvrp)
# print(geneticAlgorithm2.population)
# geneticAlgorithm2.evaluate(geneticAlgorithm2.population, cvrp)
# for i in range(len(geneticAlgorithm2.population)):
#     print (geneticAlgorithm2.population[i].fitness)


[ Individual: [5, 2, 3, 4, 0, 1],  Individual: [5, 2, 3, 4, 0, 1],  Individual: [2, 1, 0, 4, 3, 5],  Individual: [0, 3, 4, 2, 5, 1],  Individual: [3, 4, 0, 5, 1, 2],  Individual: [1, 4, 0, 5, 3, 2],  Individual: [3, 5, 4, 0, 2, 1],  Individual: [2, 4, 3, 5, 1, 0],  Individual: [2, 1, 4, 3, 5, 0],  Individual: [0, 4, 5, 1, 3, 2]]
2772.0
2772.0
3225.0
3274.0
3357.0
3446.0
3594.0
3655.0
3768.0
3948.0


In [18]:
# # TODO: Test here the methods to combine populations
# # Probamos los metodos para combinar poblaciones
# # Para este metodo combinaremos la poblacion actual population con los hijos (childs) que hemos obtenido del metodo crossover y mutation

# # Llamamos al metodo evaluate de Individual para que asigne un fitness a cada individuo de la poblacion
# for individual in gn.population:
#     individual.evaluate_first_number()

# for individual in childs:
#     individual.evaluate_first_number()
    

# # Probamos el metodo para combinar poblaciones
# new_population = gn.combine(childs)

# # Imprimimos la nueva poblacion
# print('Nueva población')
# for individual in new_population:
#     print(individual.genes)

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [19]:
# TODO: Test here the method to initialize the capacited vehicle routing problem
# # Inicializamos el vehiculo


geneticAlgorithm4 = Genetic(10, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

problem = "more_cases-low_dimension\problems\example.json"

cvrp = CVRP(problem, geneticAlgorithm4)

# Comprobamos todas las variables del problema
print(cvrp.vehicles[0])
print(cvrp.warehouse)
for i in range(len(cvrp.parcels)):
    print(cvrp.parcels[i])

print(cvrp.parcels_dict)

# Obtenemos la longitud del diccionario



suma = cvrp.search(11, 7)
print(suma)






{'id': 0, 'capacity': 20}
11
{'id': 0, 'city': 9, 'weight': 5}
{'id': 1, 'city': 5, 'weight': 7}
{'id': 2, 'city': 7, 'weight': 11}
{'id': 3, 'city': 3, 'weight': 6}
{'id': 4, 'city': 3, 'weight': 6}
{'id': 5, 'city': 8, 'weight': 14}
{0: (9, 5), 1: (5, 7), 2: (7, 11), 3: (3, 6), 4: (3, 6), 5: (8, 14)}
251.0


In [20]:
# TODO: Test here the method to solve the search problem
# Resolvemos el problema de busqueda
# print(cvrp.search(11, 9))
# Hacemos la busqueda 

In [21]:
# TODO: Test here the method to solve the capacited vehicle routing problem
# Resolvemos el problema del CVRP
problem = "more_cases-low_dimension\problems\medium.json"
# problem = "problems\competition1.json"
# for semilla in range(1,2):
    # Genetic -> (population_size, generations, selection, crossover, crossover_prob, mutation, mutation_prob, elitism, semilla, parallel)
geneticAlgorithm = Genetic(50, 100, "torneo", 'crossover', 0.9, 'mutation', 0.5, True, 1, False, True)

cvrp = CVRP(problem, algorithm = geneticAlgorithm)

solution1 = cvrp()



Problem:  more_cases-low_dimension\problems\medium.json 
Solution:   Individual: [37, 56, 92, 89, 27, 35, 71, 33, 17, 78, 38, 8, 76, 26, 95, 18, 68, 44, 99, 51, 0, 50, 53, 48, 85, 61, 30, 7, 41, 64, 49, 40, 13, 98, 34, 75, 90, 28, 65, 23, 54, 32, 91, 69, 57, 46, 72, 4, 81, 74, 39, 42, 70, 22, 87, 24, 62, 77, 84, 45, 29, 21, 96, 63, 11, 94, 31, 59, 88, 52, 25, 12, 20, 3, 10, 67, 19, 36, 16, 82, 55, 93, 73, 66, 9, 6, 58, 43, 83, 86, 47, 14, 80, 79, 1, 97, 60, 5, 15, 2] 
Stats:  {'time': 1.2503986000010627, 'searches': {'real': 254, 'total': 5485}, 'population': {'real': 51, 'total': 250}} 
Score:  38810.17000000001


In [ ]:
# Evaluar un individuo concreto
print(cvrp.evaluate([1, 4, 0, 5, 3, 2]))

3446.0


In [24]:
problem = "more_cases-low_dimension\problems\example.json"

geneticAlgorithm = Genetic(150, 1000, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

cvrp = CVRP(problem, algorithm = geneticAlgorithm)

cvrp.evaluate([1, 3, 0, 4, 2, 5])

3314.0

### 4.3. Estudio y mejora de los algoritmos

Una vez que los algoritmos han sido implementados, se debe **estudiar** su **rendimiento**. Para ello, se debe comparar la **calidad** de las **soluciones obtenidas**, así como las **diferentes estadísticas** que se consideren adecuadas (número de generaciones, tiempo de ejecución, etc.). Factores como el tamaño máximo del problema que se soporta sin un error de memoria, así como el efecto temporal de usar escenarios más complejos son otros factores a tener en cuenta. Además, se **pueden proponer** y se valorarán la incorporación de **técnicas** que **permitan acelerar** la **ejecución** de los **algoritmos**.

---

In [51]:
from concurrent.futures import ProcessPoolExecutor

def solution(semilla):

    problem = "more_cases-low_dimension\problems\example.json"
    # Genetic -> (population_size, generations, selection, crossover, crossover_prob, mutation, mutation_prob, elitism, semilla, parallel)
    geneticAlgorithm = Genetic(50, 100, "torneo", 'crossover', 0.9, 'mutation', 0.5, True, semilla, False)

    cvrp = CVRP(problem, algorithm = geneticAlgorithm)

    return cvrp()

# Crea un ejecutor con 4 procesos
with ProcessPoolExecutor(max_workers=4) as executor:
    # Aplica la función a cada valor de la semilla de forma paralela
    solutions = executor.map(solution, range(1, 4))

# print(list(solutions))



In [55]:
import multiprocessing

def solution(semilla):
    # Genetic -> (population_size, generations, selection, crossover, crossover_prob, mutation, mutation_prob, elitism, semilla, parallel)
    geneticAlgorithm = Genetic(50, 100, "torneo", 'crossover', 0.9, 'mutation', 0.5, True, semilla, False)

    cvrp = CVRP(problem, algorithm = geneticAlgorithm)

    return cvrp()

# Crea una lista de argumentos para pasar a cada proceso
args = [(problem, semilla) for semilla in range(1, 10)]

# Crea una lista de procesos
processes = [multiprocessing.Process(target=solution, args=(arg,)) for arg in args]

# Inicia cada proceso
for process in processes:
    process.start()

# Espera a que todos los procesos terminen
for process in processes:
    process.join()

# Recoge los resultados de cada proceso
results = [process.exitcode for process in processes]

print(results)


[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [25]:
# TODO: Experiment here with the small problem
# Creamos un diccionario top 10 para guardar los mejores fitness(min) de cada ejecucion
##################################################
#              EJECUCION CON HILOS               #
##################################################

top10 = []
# problem = "more_cases-low_dimension\problems\medium.json"
problem = "problems\competition1.json"

def run_ga(semilla):
    geneticAlgorithm = Genetic(250, 1000, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, semilla, False)
    cvrp = CVRP(problem, algorithm = geneticAlgorithm)
    solution = cvrp()
    print(solution)

    
    return solution


##################################################
#              EJECUCION HILOS                   #
##################################################
# # Crea una lista de hilos
# threads = []

# # Recorre la semilla y crea un hilo para cada elemento
# for i in range(20):
#     t = threading.Thread(target=run_ga, args=(i,))
#     threads.append(t)

# # Inicia todos los hilos
# for t in threads:
#     t.start()
#     t.join()

# # Espera a que todos los hilos terminen
# for t in threads:
#     t.join()


##################################################
#            EJECUCION CON PROCESOS              #
##################################################

# Crea una Pool con el número máximo de procesos posibles
with multiprocessing.Pool(None) as p:
   # Aplica la función run_ga a cada elemento de range(20) de forma asíncrona
   results = [p.apply_async(run_ga, (i,)) for i in range(2)]
   
   

   # Espera a que todos los procesos terminen
   for result in results:
       result.wait()
    
    # Mostramos los resultados





In [ ]:


    # if len(top10) < 10:
    #     top10.append(solution)
    #     top10.sort(key=lambda x: x.fitness)
    # else:
    #     for top10_fitness in top10:
    #         # Comprobamos el fitness de la solucion y lo guardamos en el diccionario si esta entre los 10 mejores
    #         if solution.fitness < top10_fitness:
    #             if len(top10) < 10:
    #                 top10.append(solution)
    #                 # Ordenamos la lista de menor a mayor
    #                 top10.sort(key=lambda x: x.fitness)
    #                 # Salimos del bucle for
    #                 break
    #             else:
    #                 top10.pop()
    #                 top10.append(solution)
    #                 # Salimos del bucle for
    #                 break


print("\n\nTop 10: ", top10)


In [27]:
# Top 10 
print("Top 10: ")
for i in range(len(top10)):
    print("Top ", i+1, ": ", top10[i], "\nFitness: " ,top10[i].fitness)
    print("\n")


Top 10: 
Top  1 :   Individual: [76, 93, 16, 57, 60, 43, 37, 7, 38, 62, 17, 59, 10, 27, 95, 61, 64, 40, 92, 77, 39, 30, 86, 73, 83, 31, 58, 19, 21, 78, 87, 80, 52, 82, 34, 65, 46, 29, 24, 36, 5, 3, 72, 23, 85, 26, 4, 6, 11, 54, 15, 0, 25, 33, 84, 88, 94, 81, 18, 67, 32, 45, 22, 56, 28, 53, 51, 1, 44, 42, 14, 75, 12, 89, 90, 9, 49, 35, 48, 98, 97, 79, 99, 68, 66, 74, 63, 2, 20, 47, 8, 70, 71, 50, 55, 41, 91, 13, 69, 96] 
Fitness:  36748.63


Top  2 :   Individual: [17, 57, 5, 64, 28, 52, 83, 81, 63, 66, 13, 3, 34, 23, 20, 89, 70, 1, 68, 48, 60, 49, 26, 0, 98, 84, 22, 21, 58, 95, 51, 65, 38, 71, 8, 42, 69, 11, 6, 92, 12, 43, 14, 36, 39, 82, 46, 77, 33, 4, 78, 61, 54, 7, 35, 94, 88, 30, 53, 96, 9, 76, 72, 75, 29, 59, 74, 85, 99, 24, 67, 80, 86, 97, 91, 45, 93, 25, 41, 27, 40, 87, 90, 50, 62, 32, 19, 15, 18, 47, 44, 16, 2, 31, 79, 73, 10, 56, 37, 55] 
Fitness:  36886.10000000002


Top  3 :   Individual: [34, 85, 97, 71, 83, 43, 42, 41, 22, 91, 21, 23, 57, 56, 74, 92, 78, 36, 37, 75, 14, 12

In [29]:
# Evaluamos el fitness 
print("Fitness: ", cvrp.evaluate([45, 58, 95, 20, 83, 29, 80, 68, 34, 89, 50, 46, 44, 99, 17, 24, 0, 77, 67, 84, 82, 69, 49, 22, 60, 21, 43, 11, 74, 65, 63, 54, 32, 92, 85, 86, 76, 35, 51, 52, 15, 14, 26, 87, 57, 3, 7, 59, 23, 30, 66, 71, 4, 2, 62, 97, 94, 48, 93, 39, 70, 6, 72, 10, 5, 33, 64, 40, 91, 78, 12, 18, 90, 41, 75, 47, 96, 36, 8, 9, 28, 25, 53, 19, 88, 27, 31, 56, 79, 37, 1, 42, 81, 13, 16, 61, 98, 38, 73, 55]))

KeyError: 45

In [ ]:
# TODO: Experiment here with the medium problem




In [ ]:
# TODO: Experiment here with the large problem

### 5. Entrega y evaluación

Al igual que la práctica anterior, esta se debe **hacer en pares**. No obstante, en **casos excepcionales** se permite realizarla **individualmente**. **La fecha límite para subir la práctica es el 18 de diciembre de 2022 a las 23:55**. Las **entrevistas y evaluaciones** se realizarán la **semana siguiente**.

Algunas consideraciones:

* **En caso de que no se haya entregado la primera práctica, o se haya sacado menos de un cuatro, se podrán entregar conjuntamente en esta fecha. No obstante, se considerará únicamente un 90% de la nota global de prácticas**.
* Está práctica supone el **70%** de la **nota** en este apartado.
* La práctica se **evaluará** mediante una **entrevista individual** con el profesorado. Las fechas de las entrevistas se publicarán con antelación.
*  Se proporcionará un **conjunto** de **casos** de **prueba preliminares** (varios mapas e instancias) que se **deben resolver correctamente**. En caso contrario, la práctica se considerará suspensa.
* La **entrevista** consistirá en una serie de **preguntas** acerca del **código**.

**Por último, para la evaluación no continua se requirirá la implementación del algoritmo de búsqueda por ascenso de colinas. Además, este se deberá utilizar para inicializar la población del algoritmo genético, en lugar de que sea aleatoria.**